In [1]:
# --- Imports & Setup ---
import os, sys, json, time
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

from sklearn.metrics import (
    roc_curve, auc, precision_recall_curve, average_precision_score,
    classification_report, confusion_matrix, brier_score_loss,
    balanced_accuracy_score, matthews_corrcoef
)
from sklearn.calibration import calibration_curve, IsotonicRegression
import joblib

ROOT = os.path.abspath("..")
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

In [2]:
# Config laden
with open(os.path.join(ROOT, "config.json"), "r") as f:
    C = json.load(f)
TICKER, START, END, INTERVAL = C["ticker"], C["start"], C["end"], C["interval"]
HORIZON, LOOKBACK = int(C["horizon"]), int(C["lookback"])
BATCH, EPOCHS = int(C["batch"]), int(C["epochs"])
SEED = int(C.get("seed", 42))

np.random.seed(SEED)
tf.random.set_seed(SEED)

RESULTS_DIR = Path(C.get("results_dir", "../results"))
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

def _latest_run_dir(results_dir: Path) -> Path:
    runs = sorted(results_dir.glob("*_lstm"), key=lambda p: p.stat().st_mtime, reverse=True)
    if not runs:
        raise FileNotFoundError("Kein RUN_DIR gefunden. Bitte Block 3 trainieren.")
    return runs[0]

RUN_DIR = _latest_run_dir(RESULTS_DIR)
print("RUN_DIR:", RUN_DIR)

RUN_DIR: ..\results\2025-10-14_16-59-43_lstm


In [3]:
# --- Artefakte laden ---
MODEL_PATH  = RUN_DIR / "model.keras"
SCALER_PATH = RUN_DIR / "scaler.joblib"
CFG_PATH    = RUN_DIR / "config.json"
assert MODEL_PATH.exists() and SCALER_PATH.exists() and CFG_PATH.exists(), "Fehlende Artefakte."

with open(CFG_PATH, "r") as f:
    RCFG = json.load(f)

# Konsistenz zur Laufzeit-Config
assert int(RCFG["horizon"]) == HORIZON and int(RCFG["lookback"]) == LOOKBACK, "Config-Mismatch."

model  = keras.models.load_model(MODEL_PATH, compile=False)  # compile=False vermeidet _loss-Deser.
scaler = joblib.load(SCALER_PATH)

# --- Daten laden ---
TRAIN_CSV = f"../data/{TICKER}_{INTERVAL}_{START}_{END}_cls_h{HORIZON}.csv"
df = pd.read_csv(TRAIN_CSV, index_col=0, parse_dates=True).sort_index()

In [4]:
# --- Feature-Spalten bestimmen ---
OHLCV = {"open","high","low","close","volume"}
candidates = [c for c in df.columns if c not in (OHLCV | {"target"})]

FEATURES = RCFG.get("features", None)
if FEATURES:
    FEATURES = [c for c in FEATURES if c in candidates]
else:
    FEATURES = candidates
assert len(FEATURES) > 0, f"Keine Features gefunden. Kandidaten: {candidates}"

# X/y
X = df[FEATURES].copy()
y = df["target"].astype(int).copy()

In [5]:
# --- Chronologische Splits (70/15/15) ---
n = len(df)
n_train = int(n * 0.70); n_val = int(n * 0.15); n_test = n - n_train - n_val
train_idx = slice(0, n_train)
val_idx   = slice(n_train, n_train + n_val)
test_idx  = slice(n_train + n_val, n)

X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
X_val,   y_val   = X.iloc[val_idx],   y.iloc[val_idx]
X_test,  y_test  = X.iloc[test_idx],  y.iloc[test_idx]

In [6]:
# --- Skalierung (wie in Block 3: nur transformieren) ---
X_train_s = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=FEATURES)
X_val_s   = pd.DataFrame(scaler.transform(X_val),   index=X_val.index,   columns=FEATURES)
X_test_s  = pd.DataFrame(scaler.transform(X_test),  index=X_test.index,  columns=FEATURES)


In [7]:
# --- Windowing ---
def make_windows(X_df: pd.DataFrame, y_ser: pd.Series, lookback: int):
    X_values = X_df.values.astype(np.float32)
    y_values = y_ser.values.astype(np.int32)
    n = len(X_df)
    xs, ys, idx_end = [], [], []
    for i in range(lookback-1, n):
        xs.append(X_values[i - lookback + 1 : i + 1])
        ys.append(y_values[i])
        idx_end.append(X_df.index[i])
    return np.stack(xs, axis=0), np.array(ys), pd.DatetimeIndex(idx_end)

Xtr_win, ytr, idx_tr = make_windows(X_train_s, y_train, LOOKBACK)
Xva_win, yva, idx_va = make_windows(X_val_s,   y_val,   LOOKBACK)
Xte_win, yte, idx_te = make_windows(X_test_s,  y_test,  LOOKBACK)

def to_ds(X, y, batch, shuffle):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(X), seed=SEED, reshuffle_each_iteration=False)
    return ds.batch(BATCH).prefetch(tf.data.AUTOTUNE)

ds_val  = to_ds(Xva_win, yva, BATCH, shuffle=False)
ds_test = to_ds(Xte_win, yte, BATCH, shuffle=False)

In [8]:
# --- Roh-Wahrscheinlichkeiten ---
y_val_proba  = model.predict(ds_val,  verbose=0).ravel()
y_test_proba = model.predict(ds_test, verbose=0).ravel()

In [9]:
# --- Kalibrierung (Isotonic auf Val) ---
iso = IsotonicRegression(out_of_bounds="clip").fit(y_val_proba, yva)
y_val_proba_cal  = iso.transform(y_val_proba)
y_test_proba_cal = iso.transform(y_test_proba)
joblib.dump(iso, RUN_DIR / "calibrator.joblib")

# --- Schwellenwahl (Val) mit Pos-Rate-Korridor ---
from sklearn.metrics import matthews_corrcoef

def choose_threshold(y_true, y_prob, strategy="max_mcc", pos_rate_bounds=(0.2, 0.8)):
    uniq = np.unique(y_prob); cand = np.r_[0.0, uniq, 1.0]
    best_t, best_s = 0.5, -1.0
    for t in cand:
        yp = (y_prob >= t).astype(int)
        pr = float(yp.mean())
        if not (pos_rate_bounds[0] <= pr <= pos_rate_bounds[1]):
            continue
        if strategy == "max_mcc":
            s = matthews_corrcoef(y_true, yp)
        else:
            s = matthews_corrcoef(y_true, yp)
        if s > best_s: best_s, best_t = float(s), float(t)
    if best_s < 0:  # Fallback
        return 0.5, 0.0
    return best_t, best_s

thr, score_val = choose_threshold(yva, y_val_proba_cal, strategy="max_mcc", pos_rate_bounds=(0.2,0.8))
print(f"Gewählter Threshold (Val, kalibriert): {thr:.3f} | MCC_val={score_val:.3f}")

Gewählter Threshold (Val, kalibriert): 0.500 | MCC_val=0.000


In [10]:
# --- Test-Evaluation (kalibriert @ thr) ---
y_test_pred = (y_test_proba_cal >= thr).astype(int)

cm   = confusion_matrix(yte, y_test_pred)
rep  = classification_report(yte, y_test_pred, digits=3, output_dict=True)
fpr, tpr, _ = roc_curve(yte, y_test_proba_cal)
roc_auc = auc(fpr, tpr)
prec, rec, _ = precision_recall_curve(yte, y_test_proba_cal)
ap = average_precision_score(yte, y_test_proba_cal)
brier_raw = brier_score_loss(yte, y_test_proba)       # vor Kal.
brier_cal = brier_score_loss(yte, y_test_proba_cal)   # nach Kal.
bal_acc = balanced_accuracy_score(yte, y_test_pred)
mcc     = matthews_corrcoef(yte, y_test_pred)

print("Confusion matrix (test):\n", cm)
print("MCC:", round(mcc,3), "BalancedAcc:", round(bal_acc,3), "AUROC:", round(roc_auc,3), "AUPRC:", round(ap,3))

Confusion matrix (test):
 [[  4 204]
 [  6 238]]
MCC: -0.018 BalancedAcc: 0.497 AUROC: 0.486 AUPRC: 0.534


In [11]:
# --- Bootstrap-CI fürs MCC (Blocksampling) ---
rng = np.random.default_rng(SEED)
def block_bootstrap_mcc(y_true, y_prob, threshold, n=300, block=LOOKBACK):
    idx = np.arange(len(y_true))
    scores = []
    for _ in range(n):
        starts = rng.integers(0, max(1, len(idx)-block+1), size=max(1, len(idx)//block))
        bs = np.concatenate([np.arange(s, min(s+block, len(idx))) for s in starts])
        yp = (y_prob[bs] >= threshold).astype(int)
        scores.append(matthews_corrcoef(y_true[bs], yp))
    return np.percentile(scores, [2.5, 50, 97.5]).astype(float)
mcc_ci = block_bootstrap_mcc(yte, y_test_proba_cal, thr, n=300, block=LOOKBACK)
print("MCC Bootstrap CI [2.5,50,97.5]:", [round(x,3) for x in mcc_ci])


MCC Bootstrap CI [2.5,50,97.5]: [np.float64(-0.059), np.float64(-0.008), np.float64(0.056)]


In [12]:
# --- Plots ---
FIG_DIR = RUN_DIR / "figures"
FIG_DIR.mkdir(exist_ok=True, parents=True)

plt.figure(figsize=(6,4)); plt.plot(fpr, tpr, label=f"AUC={roc_auc:.3f}")
plt.plot([0,1],[0,1],"--"); plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("ROC (Test)")
plt.legend(); plt.tight_layout(); plt.savefig(FIG_DIR / "roc_test.png", dpi=160); plt.close()

plt.figure(figsize=(6,4)); plt.plot(rec, prec, label=f"AP={ap:.3f}")
plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("Precision-Recall (Test)")
plt.legend(); plt.tight_layout(); plt.savefig(FIG_DIR / "pr_test.png", dpi=160); plt.close()

prob_true, prob_pred = calibration_curve(yte, y_test_proba_cal, n_bins=10, strategy="quantile")
plt.figure(figsize=(6,4)); plt.plot([0,1],[0,1],"--"); plt.plot(prob_pred, prob_true, marker="o")
plt.xlabel("Vorhergesagt"); plt.ylabel("Tatsächlich"); plt.title("Kalibrierung (Test)")
plt.tight_layout(); plt.savefig(FIG_DIR / "calibration_test.png", dpi=160); plt.close()

plt.figure(figsize=(4.8,4.2))
plt.imshow(cm, interpolation="nearest"); plt.title("Confusion Matrix (Test)"); plt.colorbar()
ticks = np.arange(2); plt.xticks(ticks, ["0","1"]); plt.yticks(ticks, ["0","1"])
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j], ha="center", va="center")
plt.xlabel("Predicted"); plt.ylabel("True")
plt.tight_layout(); plt.savefig(FIG_DIR / "cm_test.png", dpi=160); plt.close()

plt.figure(figsize=(6,4))
plt.hist(y_test_proba, bins=30, alpha=0.6, label="raw")
plt.hist(y_test_proba_cal, bins=30, alpha=0.6, label="calibrated (isotonic)")
plt.axvline(thr, linestyle="--", label=f"thr={thr:.3f}")
plt.title("P(y=1) Verteilung (Test) – roh vs. kalibriert")
plt.legend(); plt.tight_layout(); plt.savefig(FIG_DIR / "proba_hist_raw_vs_cal.png", dpi=160); plt.close()

In [13]:
# --- Predictions-CSV (Test, kalibriert) ---
preds_test = pd.DataFrame({
    "timestamp": idx_te,
    "y_true": yte,
    "y_proba_raw": y_test_proba,
    "y_proba_cal": y_test_proba_cal,
    "y_pred": y_test_pred,
}).set_index("timestamp")
preds_test.to_csv(RUN_DIR / "preds_test.csv")

In [14]:
# --- Einfache Handels-Strategien (Entry @t und @t+1) ---
close = df["close"].copy()
fwd_logret = (np.log(close.shift(-HORIZON)) - np.log(close)).reindex(idx_te)

signals_t  = (preds_test["y_proba_cal"] >= thr).astype(int).reindex(idx_te)
signals_t1 = signals_t.shift(1).fillna(0)

strategy_logret_t  = (signals_t  * fwd_logret).fillna(0)
strategy_logret_t1 = (signals_t1 * fwd_logret).fillna(0)

equity_t  = strategy_logret_t.cumsum().apply(np.exp)
equity_t1 = strategy_logret_t1.cumsum().apply(np.exp)

bh_logret = (np.log(close.reindex(idx_te)) - np.log(close.reindex(idx_te).iloc[0])).fillna(0)
bh_equity = np.exp(bh_logret)

def _sharpe(logrets, periods_per_year=252):
    if len(logrets) < 2: return float("nan")
    mu = logrets.mean() * periods_per_year
    sigma = logrets.std(ddof=1) * np.sqrt(periods_per_year)
    return float(mu / (sigma + 1e-12))

def _cagr(equity_series, periods_per_year=252):
    if len(equity_series) < 2: return float("nan")
    T = len(equity_series) / periods_per_year
    return float((equity_series.iloc[-1] / equity_series.iloc[0])**(1.0/T) - 1.0)

backtest = {
    "n_trades": int(signals_t.sum()),
    "avg_holding_h": HORIZON,
    "strategy_t": {
        "CAGR": _cagr(equity_t),
        "Sharpe": _sharpe(strategy_logret_t.dropna()),
        "final_equity": float(equity_t.iloc[-1]),
    },
    "strategy_t1": {
        "CAGR": _cagr(equity_t1),
        "Sharpe": _sharpe(strategy_logret_t1.dropna()),
        "final_equity": float(equity_t1.iloc[-1]),
    },
    "buy_hold": {
        "CAGR": _cagr(bh_equity),
        "final_equity": float(bh_equity.iloc[-1]),
    },
}

plt.figure(figsize=(8,4))
plt.plot(equity_t.index, equity_t.values,   label="Entry@t (optimistisch)")
plt.plot(equity_t1.index, equity_t1.values, label="Entry@t+1 (konservativ)")
plt.plot(bh_equity.index, bh_equity.values, label="Buy & Hold", linestyle="--")
plt.title(f"Equity Curves (H={HORIZON})")
plt.legend(); plt.tight_layout()
plt.savefig(FIG_DIR / "equity_curves_t_vs_t1.png", dpi=160); plt.close()

plt.figure(figsize=(6,4))
plt.boxplot([fwd_logret[signals_t==0].dropna(), fwd_logret[signals_t==1].dropna()],
            tick_labels=["Signal=0","Signal=1"])
plt.title("Forward Log-Return nach Signal")
plt.tight_layout(); plt.savefig(FIG_DIR / "forward_returns_by_signal.png", dpi=160); plt.close()

In [15]:
# --- Vor/Nach: Histogramm + Brier ---
calib_name = "isotonic"  # oder aus deiner Pipeline lesen

brier_raw = brier_score_loss(yte, y_test_proba)
brier_cal = brier_score_loss(yte, y_test_proba_cal)

plt.figure(figsize=(6,4))
plt.hist(y_test_proba,      bins=30, alpha=0.6, label="raw")
plt.hist(y_test_proba_cal,  bins=30, alpha=0.6, label=f"calibrated ({calib_name})")
plt.axvline(thr, linestyle="--", label=f"thr={thr:.3f}")
plt.title("P(y=1) – raw vs. calibrated (Test)")
plt.legend(); plt.tight_layout()
plt.savefig(FIG_DIR / "proba_hist_raw_vs_cal.png", dpi=160); plt.close()

print(f"Brier raw={brier_raw:.4f} → calibrated={brier_cal:.4f}")


Brier raw=0.2523 → calibrated=0.2571


In [16]:
# --- Ergebnisse schreiben ---
out = {
    "config": RCFG,
    "features_used": FEATURES,
    "threshold_selection": {
        "strategy": "max_mcc_with_pos_rate_bounds",
        "threshold": float(thr),
        "pos_rate_bounds": [0.2, 0.8],
        "val_mcc": float(score_val),
    },
    "calibration": {"method": "isotonic", "path": str(RUN_DIR / "calibrator.joblib"),
                    "brier_raw": float(brier_raw), "brier_cal": float(brier_cal)},
    "metrics": {
        "test": {
            "roc_auc": float(roc_auc),
            "auprc": float(ap),
            "brier": float(brier_cal),
            "balanced_accuracy": float(bal_acc),
            "mcc": float(mcc),
            "mcc_bootstrap_ci": [float(mcc_ci[0]), float(mcc_ci[1]), float(mcc_ci[2])],
            "confusion_matrix": cm.tolist(),
            "report": rep,
        }
    },
    "diagnostics": {
        "val_pos_rate_cal": float(y_val_proba_cal.mean()),
        "test_pos_rate_cal": float(y_test_proba_cal.mean()),
        "test_pred_pos_rate_at_thr": float((y_test_proba_cal >= thr).mean()),
    },
    "backtest": backtest,
}
with open(RUN_DIR / "evaluation.json", "w") as f:
    json.dump(out, f, indent=2)

print("\nBlock 4 abgeschlossen.")
print("Artefakte:")
print(" -", RUN_DIR / "preds_test.csv")
print(" -", RUN_DIR / "evaluation.json")
print(" -", RUN_DIR / "figures")


Block 4 abgeschlossen.
Artefakte:
 - ..\results\2025-10-14_16-59-43_lstm\preds_test.csv
 - ..\results\2025-10-14_16-59-43_lstm\evaluation.json
 - ..\results\2025-10-14_16-59-43_lstm\figures
